# DePaul University - CSC 594 - Topics in AI: Text Mining and Analytics
# Professor Noriko Tomuro
# Final Project - Code and Processing Results
# Student: Ricardo Barros Lourenço

In [1]:
from nltk import *
from nltk.corpus import stopwords 
import pandas as pd
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, pairwise, mean_squared_error
from sklearn import decomposition, preprocessing, linear_model, cross_validation
import numpy as np
import re
from multiprocessing import *
from collections import Counter
import string
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("./train.csv")

In [3]:
#Remove any register with one or more missing values.
df_ETL = df.dropna(how='any')
#df_ETL.count()
#list(df_ETL.columns.values)

In [4]:
df_ETL[:5]

,id,tweet,state,location,s1,s2,s3,s4,s5,w1,...,k6,k7,k8,k9,k10,k11,k12,k13,k14,k15
0,1,Jazz for a Rainy Afternoon: {link},oklahoma,Oklahoma,0,0,1.0,0.000,0.000,0.800,...,0,0.000,0,0.000,1,0,0,0.000,0,0
1,2,RT: @mention: I love rainy days.,florida,Miami-Ft. Lauderdale,0,0,0.0,1.000,0.000,0.196,...,0,0.000,0,0.000,1,0,0,0.000,0,0
3,6,Preach lol! :) RT @mention: #alliwantis this t...,minnesota,Minneapolis-St. Paul,0,0,0.0,1.000,0.000,1.000,...,0,0.604,0,0.196,0,0,0,0.201,0,0
4,9,@mention good morning sunshine,rhode island,Purgatory,0,0,0.0,0.403,0.597,1.000,...,0,0.000,0,0.000,0,0,0,1.000,0,0
5,10,Rather be storm chasing.,district of columbia,"Washington, DC",0,0,0.6,0.000,0.400,0.000,...,0,0.000,0,0.000,0,0,1,0.000,0,0


In [5]:
# Normalize attributes s, w, and k and round to the closer integer.
keep = df_ETL[['id','tweet','state','location']]
norm = df_ETL[['s1','s2','s3','s4','s5','w1','w2','w3','w4','k1','k2','k3','k4','k5','k6','k7','k8','k9','k10','k11','k12','k13','k14','k15']]
min_max_scaler = sk.preprocessing.MinMaxScaler().fit(norm)
norm = pd.DataFrame(min_max_scaler.transform(norm))
norm.columns = ['s1','s2','s3','s4','s5','w1','w2','w3','w4','k1','k2','k3','k4','k5','k6','k7','k8','k9','k10','k11','k12','k13','k14','k15']
norm = norm.apply(np.around)
df_ETL = pd.concat([keep, norm], axis=1)

In [6]:
df_ETL[:5]

,id,tweet,state,location,s1,s2,s3,s4,s5,w1,...,k6,k7,k8,k9,k10,k11,k12,k13,k14,k15
0,1,Jazz for a Rainy Afternoon: {link},oklahoma,Oklahoma,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,2,RT: @mention: I love rainy days.,florida,Miami-Ft. Lauderdale,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,NaN,NaN,NaN,NaN,0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
3,6,Preach lol! :) RT @mention: #alliwantis this t...,minnesota,Minneapolis-St. Paul,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
4,9,@mention good morning sunshine,rhode island,Purgatory,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [7]:
def preprocess(document):
    document = str(document)
    lowers = document.lower()
    sentences = lowers.translate(None, string.punctuation)
    return sentences

In [8]:
# Apply preprocessing using parallelization
pool = Pool()
df_ETL["tweet"] = pool.map(preprocess, df_ETL["tweet"])
df_ETL = df_ETL.dropna(axis=0, how='any')

In [9]:
tweets = df_ETL["tweet"]

In [10]:
tweets

0                         jazz for a rainy afternoon  link
1                             rt mention i love rainy days
3        preach lol  rt mention alliwantis this type of...
4                            mention good morning sunshine
5                                  rather be storm chasing
6        weather  154 pm  610f feels 60f 2998 humidity ...
7        father son holy ghost amen hope i come out ali...
8        mild monday best weather day of the week in ok...
9        with the snow forecast for tahoe this weekend ...
10       no signs of the appocolypse yet sunshine and w...
11       outside 627 f humidity 71 wind 121 11 mph pres...
12       mention another storm  at least were getting r...
14       can already tell its going to be a tough scori...
15       a lil chilly all good to words for that call o...
16       loving the sugarland concert  even though im s...
18       today was windy af i dont wanna go meet up w m...
19       mention damnhomie perfect weather over here wh.

In [11]:
tfidf_vectorizer = TfidfVectorizer(min_df=0, vocabulary=None)
tfidf_matrix = tfidf_vectorizer.fit_transform(tweets)

In [12]:
factor_matrix = tfidf_matrix
#factor_matrix.shape

In [13]:
label = "s1"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  s1  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.36256376]]
Mean square error of predictions on training:  0.00815924096422
Cosine similarity of predictions on training:  [[ 0.07050809]]
Mean square error of predictions on training:  0.00953531775287


In [14]:
label = "s2"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  s2  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.5339913]]
Mean square error of predictions on training:  0.157100370564
Cosine similarity of predictions on training:  [[ 0.45649852]]
Mean square error of predictions on training:  0.172600965991


In [15]:
label = "s3"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  s3  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.57487834]]
Mean square error of predictions on training:  0.17928592786
Cosine similarity of predictions on training:  [[ 0.50172722]]
Mean square error of predictions on training:  0.198293383659


In [16]:
label = "s4"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  s4  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.54060277]]
Mean square error of predictions on training:  0.159997568263
Cosine similarity of predictions on training:  [[ 0.46448993]]
Mean square error of predictions on training:  0.17751538096


In [17]:
label = "s5"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  s5  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.46991378]]
Mean square error of predictions on training:  0.113367347873
Cosine similarity of predictions on training:  [[ 0.37629825]]
Mean square error of predictions on training:  0.127938261081


In [18]:
label = "w1"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  w1  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.88344279]]
Mean square error of predictions on training:  0.167018851532
Cosine similarity of predictions on training:  [[ 0.86051332]]
Mean square error of predictions on training:  0.193434004848


In [19]:
label = "w2"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  w2  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.4034694]]
Mean square error of predictions on training:  0.062681253066
Cosine similarity of predictions on training:  [[ 0.25605018]]
Mean square error of predictions on training:  0.0710683285426


In [20]:
label = "w3"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  w3  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.4125798]]
Mean square error of predictions on training:  0.0713068193365
Cosine similarity of predictions on training:  [[ 0.30030455]]
Mean square error of predictions on training:  0.0888833241971


In [21]:
label = "w4"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  w4  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.37585818]]
Mean square error of predictions on training:  0.0403379644448
Cosine similarity of predictions on training:  [[ 0.20712021]]
Mean square error of predictions on training:  0.0461987779495


In [22]:
label = "k1"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k1  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.36946729]]
Mean square error of predictions on training:  0.0179686719399
Cosine similarity of predictions on training:  [[ 0.12285882]]
Mean square error of predictions on training:  0.0222598876411


In [23]:
label = "k2"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k2  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.41895625]]
Mean square error of predictions on training:  0.0757825809273
Cosine similarity of predictions on training:  [[ 0.27806414]]
Mean square error of predictions on training:  0.0810590196241


In [24]:
label = "k3"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k3  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.39010357]]
Mean square error of predictions on training:  0.00341814772096
Cosine similarity of predictions on training:  [[ 0.02143839]]
Mean square error of predictions on training:  0.00281965874134


In [25]:
label = "k4"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k4  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.44479568]]
Mean square error of predictions on training:  0.0957667696511
Cosine similarity of predictions on training:  [[ 0.33818397]]
Mean square error of predictions on training:  0.107129104


In [26]:
label = "k5"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k5  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.37190634]]
Mean square error of predictions on training:  0.0329126001855
Cosine similarity of predictions on training:  [[ 0.19045296]]
Mean square error of predictions on training:  0.0419778288215


In [27]:
label = "k6"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k6  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.38692027]]
Mean square error of predictions on training:  0.00125893776443
Cosine similarity of predictions on training:  [[ 0.00817133]]
Mean square error of predictions on training:  0.00209758942512


In [28]:
label = "k7"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k7  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.57934981]]
Mean square error of predictions on training:  0.182899159084
Cosine similarity of predictions on training:  [[ 0.52128612]]
Mean square error of predictions on training:  0.20417983961


In [29]:
label = "k8"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k8  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.39926488]]
Mean square error of predictions on training:  0.00148529397989
Cosine similarity of predictions on training:  [[ 0.03352883]]
Mean square error of predictions on training:  0.00278145856612


In [30]:
label = "k9"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k9  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.36400694]]
Mean square error of predictions on training:  0.0254738991996
Cosine similarity of predictions on training:  [[ 0.14905313]]
Mean square error of predictions on training:  0.0241954254966


In [31]:
label = "k10"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k10  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.42846342]]
Mean square error of predictions on training:  0.0847661602487
Cosine similarity of predictions on training:  [[ 0.30566032]]
Mean square error of predictions on training:  0.0913343984262


In [32]:
label = "k11"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k11  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.37052505]]
Mean square error of predictions on training:  0.0313457780886
Cosine similarity of predictions on training:  [[ 0.15360364]]
Mean square error of predictions on training:  0.0350430510763


In [33]:
label = "k12"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k12  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.47618673]]
Mean square error of predictions on training:  0.119136944626
Cosine similarity of predictions on training:  [[ 0.3757311]]
Mean square error of predictions on training:  0.128014170088


In [34]:
label = "k13"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k13  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.46969923]]
Mean square error of predictions on training:  0.114053374503
Cosine similarity of predictions on training:  [[ 0.37671485]]
Mean square error of predictions on training:  0.130278352943


In [35]:
label = "k14"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k14  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.35913445]]
Mean square error of predictions on training:  0.0179265219602
Cosine similarity of predictions on training:  [[ 0.11212666]]
Mean square error of predictions on training:  0.0208136372975


In [36]:
label = "k15"
target = np.array(df_ETL[label])
#target = target.reshape(57631,1)
#target.shape

factor_matrix_train, factor_matrix_test, target_train, target_test = cross_validation.train_test_split(factor_matrix, target, test_size=0.10, random_state=10)
ridge_cv = linear_model.RidgeCV(cv=10).fit(factor_matrix_train, target_train)
eval_train = ridge_cv.predict(factor_matrix_train)
eval_test = ridge_cv.predict(factor_matrix_test)

print 'Ridge Regression Results for variable ',label,' using 10-fold cross validation :'

#Cosine Similarity Among Prediction and Labels On Training
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_train,target_train)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_train, eval_train, sample_weight=None, multioutput='uniform_average')

#Cosine Similarity Among Prediction and Labels On Testing
print 'Cosine similarity of predictions on training: ', pairwise.cosine_similarity(eval_test,target_test)

# MSE
print 'Mean square error of predictions on training: ',mean_squared_error(target_test, eval_test, sample_weight=None, multioutput='uniform_average')

Ridge Regression Results for variable  k15  using 10-fold cross validation :
Cosine similarity of predictions on training:  [[ 0.38245046]]
Mean square error of predictions on training:  0.0471115319845
Cosine similarity of predictions on training:  [[ 0.2118193]]
Mean square error of predictions on training:  0.0532943580958
